# Local Notebook

In [ ]:
"""
_summary_

"""

from google.cloud import bigquery
import pandas as pd
import pandas_gbq

## Instalar paquetes
Podemos utilizar funciones mágicas de los Jupyter Notebooks (ipython) para
instalar los paquetes necesarios, ya que asi nos aseguramos que estos sean
instalados en el ambiente correspondiente.

In [ ]:
%pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]'
%pip install ipywidgets

In [ ]:
%pip install --upgrade pandas

## Consultar en Bigquery

Para hacer consultas en BiQuery y guardarlas en un dataframe en python podemos
emplear 3 métodos que prácticamente son lo mismo:

1. Llamadas a BigQuery utilizando la API de python:
   Primero creamos el objeto cliente de BigQuery usando su constructor
   Client() luego usamos métodos de ese cliente para lo necesario, en
   este caso utilizamos el método query_and_wait y luego al iterador
   devuelto le aplicamos el método to_dataframe.

2. Módulo pandas_gbq:
   El cual proporciona una envoltura para el servicio web de análisis
   BigQuery de Google, en este caso utilizamos la función read_gbq del módulo.

3. Line magics o Cell magics
   Acá las lineas o celdas de magia son funciones de ipython que se pueden
   llamar con un estilo similar al sintaxis de la linea de comandos, si la
   función no es predefinida debemos cargarla si estamos en nuestro entorno
   local utilizando el nombre de la extensión.


### Alternativas:

Se definen las variables a utilizar en las alternativas como el string de
la consulta y otras según sea necesario para el tipo de consulta.

En este caso solamente definimos el string de la consulta.

In [ ]:
SQL = """
SELECT
    *
FROM bigquery-manu-407202.dsongcp.flights
"""

#### Alternativa 1 - Llamadas a BigQuery

Creamos el objeto cliente y ejecutamos la consulta y conversión del objeto

In [ ]:
client = bigquery.Client()
df_bq: pd.DataFrame = client.query_and_wait(SQL).to_dataframe()

In [ ]:
df_bq.info()

#### Alternativa 2 - Pandas Google Big Query

Utilizamos read_gbq para leer la consulta directamente en un DataFrame con
la opción de use_bqstorage_api en True para utilizar la API de
BigQuery Storage y así obtener resultados grandes en menor tiempo.

In [ ]:
# pyright: reportAssignmentType = false
df_bq_pandas: pd.DataFrame = pandas_gbq.read_gbq(SQL, use_bqstorage_api=True)

#### Alternativa 3 - Jupyter Magic

El sintaxis de la celda mágica es el siguiente:

```
%%bigquery [<destination_var>] [--project <project>] [--use_legacy_sql]
           [--verbose] [--params <params>] <query>
```

Obs. si nos encontramos en un ambiente local, antes de ejecutar la celda
debemos ejecutar la linea mágica load_ext la cual carga las funciones
mágicas por su nombre de módulo.

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df_bq_magic
SELECT
    *
FROM bigquery-manu-407202.dsongcp.flights

Descargamos el esquema de la tabla flights y borramos los campos extras
que describen la tabla.

In [ ]:
!bq show --format=prettyjson dsongcp.flights > flights_schema.json

### Leer archivos descargados desde la consola

Leemos los archivos descargados desde la consola de google cloud platform
para comparar los formatos y que debemos cambiar.

Obs. dependiendo del motor para leer los archivos ya sea `ujson` o `pyarrow`
los tipos de datos serán distintos.

In [ ]:
# import pandas as pd
df_console_json = pd.read_json("flights/flights.json",
                               orient="records",
                               lines=True,
                               engine="pyarrow"
                               )
df_console_json.info()

Prueba archivo json pequeño

In [ ]:
df_console_json_fl = pd.read_json("flights/flights_few_lines.json",
                                  lines=True,
                                  engine="pyarrow"
                                  )
df_console_json_fl.info()

In [ ]:
df_console_json_fl = pd.read_json("flights/flights_few_lines.json",
                                  orient="records",
                                  lines=True
                                  )
df_console_json_fl.info()

Prueba archivo csv pequeño

In [ ]:
df_console_csv_fl = pd.read_csv("flights/flights_few_lines.csv")
df_console_csv_fl.info()

In [ ]:
df_console_csv = pd.read_csv("flights/flights.csv")
df_console_csv.info()

In [ ]:

df_bq["FL_DATE"] = pd.to_datetime(df_bq["FL_DATE"])
df_bq["FL_DATE"] = df_bq["FL_DATE"].dt.strftime('%Y-%m-%d')

# Test
not_string_cols = ['FL_DATE', 'DEP_DELAY', 'TAXI_OUT',
                   'TAXI_IN', 'ARR_DELAY', 'CANCELLED', 'DIVERTED']
string_cols = [col for col in df_bq.columns if col not in not_string_cols]
for col in string_cols:
    df_bq[col] = df_bq[col].astype(str)

# Completar con ceros a la izquierda las columnas con formato "hhmm"
for col in ["CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF", "WHEELS_ON", "CRS_ARR_TIME", "ARR_TIME"]:
    df_bq[col] = df_bq[col].str.zfill(4)

# Eliminamos los vuelos cancelados y desviados con fines de desarrollo
df_bq = df_bq.loc[~df_bq["DIVERTED"] & ~df_bq["CANCELLED"]]

In [ ]:

json = df_bq.to_json(
    "flights_2024.json",
    orient="records",
    lines=True,
)

In [ ]:
df_csv = pd.read_csv("airports_2024.csv")
airports_tz = df_csv.iloc[:, [0, 21, 26]]
airports_tz

### dic

dic concat

In [5]:
flights_schema = {
    "fields": [
        {"mode": "NULLABLE", "name": "FL_DATE", "type": "STRING"},
        {"mode": "NULLABLE", "name": "UNIQUE_CARRIER", "type": "STRING"},
        {"mode": "NULLABLE", "name": "ORIGIN_AIRPORT_SEQ_ID", "type": "STRING"},
        {"mode": "NULLABLE", "name": "ORIGIN", "type": "STRING"},
        {"mode": "NULLABLE", "name": "DEST_AIRPORT_SEQ_ID", "type": "STRING"},
        {"mode": "NULLABLE", "name": "DEST", "type": "STRING"},
        {"mode": "NULLABLE", "name": "CRS_DEP_TIME", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "DEP_TIME", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "DEP_DELAY", "type": "FLOAT"},
        {"mode": "NULLABLE", "name": "TAXI_OUT", "type": "FLOAT"},
        {"mode": "NULLABLE", "name": "WHEELS_OFF", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "WHEELS_ON", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "TAXI_IN", "type": "FLOAT"},
        {"mode": "NULLABLE", "name": "CRS_ARR_TIME", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "ARR_TIME", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "ARR_DELAY", "type": "FLOAT"},
        {"mode": "NULLABLE", "name": "CANCELLED", "type": "BOOLEAN"},
        {"mode": "NULLABLE", "name": "DIVERTED", "type": "BOOLEAN"},
        {"mode": "NULLABLE", "name": "DISTANCE", "type": "STRING"},
        {"mode": "NULLABLE", "name": "DEP_AIRPORT_LAT", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "DEP_AIRPORT_LON", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "DEP_AIRPORT_TZOFFSET", "type": "FLOAT"},
        {"mode": "NULLABLE", "name": "ARR_AIRPORT_LAT", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "ARR_AIRPORT_LON", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "ARR_AIRPORT_TZOFFSET", "type": "FLOAT"},
    ]
}

In [6]:
events_schema = {
    "fields": [
        # Events
        {"mode": "NULLABLE", "name": "EVENT_TYPE", "type": "STRING"},
        {"mode": "NULLABLE", "name": "EVENT_TIME", "type": "TIMESTAMP"},
        {"mode": "NULLABLE", "name": "EVENT_DATA", "type": "STRING"},
    ]
}

In [3]:
# Reasignar la combinación de ambos esquemas a events_schema
events_schema = {
    "fields": flights_schema["fields"] + events_schema["fields"]
}

In [7]:
# Extender la lista de campos en events_schema directamente con los campos de flights_schema
events_schema["fields"][:0] = flights_schema["fields"]

In [8]:
events_schema

{'fields': [{'mode': 'NULLABLE', 'name': 'FL_DATE', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'UNIQUE_CARRIER', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'ORIGIN_AIRPORT_SEQ_ID', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'ORIGIN', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'DEST_AIRPORT_SEQ_ID', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'DEST', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'CRS_DEP_TIME', 'type': 'TIMESTAMP'},
  {'mode': 'NULLABLE', 'name': 'DEP_TIME', 'type': 'TIMESTAMP'},
  {'mode': 'NULLABLE', 'name': 'DEP_DELAY', 'type': 'FLOAT'},
  {'mode': 'NULLABLE', 'name': 'TAXI_OUT', 'type': 'FLOAT'},
  {'mode': 'NULLABLE', 'name': 'WHEELS_OFF', 'type': 'TIMESTAMP'},
  {'mode': 'NULLABLE', 'name': 'WHEELS_ON', 'type': 'TIMESTAMP'},
  {'mode': 'NULLABLE', 'name': 'TAXI_IN', 'type': 'FLOAT'},
  {'mode': 'NULLABLE', 'name': 'CRS_ARR_TIME', 'type': 'TIMESTAMP'},
  {'mode': 'NULLABLE', 'name': 'ARR_TIME', 'type': 'TIMESTAMP'},
  {'mode': 

In [ ]:
# Combinar los dos diccionarios
combined_fields_1 = {
    "fields": flights_schema["fields"] + events_schema["fields"]
}
combined_fields_1

In [ ]:
# Combinar los dos diccionarios
combined_schema_1 = {"fields": flights_schema["fields"].copy()}
combined_schema_1["fields"].extend(events_schema["fields"])
combined_schema_1

In [ ]:
# Crear una copia del flights_schema para no modificar el original
combined_schema_2 = flights_schema.copy()
# Extender los campos del esquema de vuelos con los campos del esquema de eventos
combined_schema_2["fields"].extend(events_schema["fields"])
# Actualizar events_schema con el nuevo esquema combinado
events_schema = combined_schema_2
events_schema